# Gradient Approach

In [2]:
import os

import numpy as np
from matplotlib import pyplot as plt
from utils.edge_extraction import *
import cv2
from utils.feature_extraction import *
from utils.image_conversion import *

In [3]:
image_green1 = cv2.imread("data/5.35.1.png", cv2.IMREAD_UNCHANGED)
image_green2 = cv2.imread("data/5.34.2.png", cv2.IMREAD_UNCHANGED)
image_yellow = cv2.imread("data/5.33.3.png", cv2.IMREAD_UNCHANGED)

working_region_image_green1 = extract_working_region(image_green1, threshold=10)
working_region_image_green2 = extract_working_region(image_green2, threshold=10)
working_region_image_yellow = extract_working_region(image_yellow, threshold=10)

fig, axes = plt.subplots(3, 2, figsize=(15, 5))
images = [image_green1, working_region_image_green1, image_green2, working_region_image_green2, image_yellow, working_region_image_yellow]
titles = ["Original Image", "Working region"]

for i, ax_row in enumerate(axes):
    for j, ax in enumerate(ax_row):
        ax.imshow(images[i * 2 + j])
        ax.set_title(titles[j])

plt.tight_layout()
plt.show()

In [4]:
def get_only_valid_pixels(input_image: np.ndarray) -> np.ndarray:
    # Extract pixels with alpha channel value different from 0
    valid_pixels = input_image[input_image[:, :, 3] != 0]

    # Reshape to (num_pixels, 4)
    reshaped_image = valid_pixels.reshape((-1, 4))

    # Determine the dimensions of the reshaped image
    num_pixels = reshaped_image.shape[0]
    width = int(np.sqrt(num_pixels))
    height = (num_pixels + width - 1) // width

    # Pad the reshaped image to make it rectangular
    padded_image = np.zeros((height * width, 4), dtype=np.uint8)
    padded_image[:num_pixels, :] = reshaped_image

    # Reshape the padded image to the desired dimensions
    final_image = padded_image.reshape((height, width, 4))

    # Split channel
    b, g, r, a = cv2.split(final_image)
    rgb_image = cv2.merge((b, g, r))

    return rgb_image

In [5]:
working_region_image_green1 = get_only_valid_pixels(working_region_image_green1)
working_region_image_green2 = get_only_valid_pixels(working_region_image_green2)
working_region_image_yellow = get_only_valid_pixels(working_region_image_yellow)

working_region_image_green1 = cv.cvtColor(working_region_image_green1, cv.COLOR_BGR2HSV)
working_region_image_green2 = cv.cvtColor(working_region_image_green2, cv.COLOR_BGR2HSV)
working_region_image_yellow = cv.cvtColor(working_region_image_yellow, cv.COLOR_BGR2HSV)

max_x = max(working_region_image_green1.shape[0], working_region_image_green2.shape[0], working_region_image_yellow.shape[0])
max_y = max(working_region_image_green1.shape[1], working_region_image_green2.shape[1], working_region_image_yellow.shape[1])

working_region_image_green1 = cv2.resize(working_region_image_green1, (max_x, max_y))
working_region_image_green2 = cv2.resize(working_region_image_green2, (max_x, max_y))
working_region_image_yellow = cv2.resize(working_region_image_yellow, (max_x, max_y))

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
images = [working_region_image_green1, working_region_image_green2,working_region_image_yellow]
titles = ["Working region"]

for i, ax in enumerate(axes):
    ax.imshow(images[i])
    ax.set_title(titles[0])

plt.show()

In [6]:
from utils.clustering import create_dataset, compute_jacobians_dist_matrix

X = create_dataset(img_dir="./data", threshold=5)
dist_matrix = compute_jacobians_dist_matrix(X)
dist_matrix.shape

In [7]:
dist_matrix

In [8]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=100, min_samples=5, metric="precomputed").fit(dist_matrix)
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

In [9]:
from utils.clustering import create_cluster_dirs

create_cluster_dirs(data_dir="./data", output_dir="clusters/jacobians", labels=labels)

In [11]:
img = cv.imread("data/5.35.3.png", cv.IMREAD_UNCHANGED)
rgb_img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title("Original Image")
plt.subplot(1, 2, 2)
plt.imshow(rgb_img)
plt.title("RGB image")
plt.show()

In [16]:
b, g, r,a = cv.split(img)
rgba_img = np.dstack((r, g, b, a))

plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title("Original Image")
plt.subplot(1, 2, 2)
plt.imshow(rgba_img)
plt.title("RGBA image")
plt.show()

In [12]:
print(img.shape)
print(rgb_img.shape)

In [13]:
img

In [14]:
rgb_img